# imports 

In [1]:
# Importing needed dependencies :
import pandas as pd
import numpy as np
import string
from string import digits
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Input, Embedding, TimeDistributed, Flatten, Dropout
from keras.callbacks import ModelCheckpoint
import re

Using TensorFlow backend.


# open fr text file

In [2]:
# read txtfile  :
file_name = 'fra.txt'
lines = pd.read_csv(file_name, names=['en', 'de'], encoding='utf-8-sig', sep='\t')

# shit 1

In [3]:
# Convert text to lowercase :
lines.en=lines.en.apply(lambda x: x.lower())
lines.de=lines.de.apply(lambda x: x.lower())

# Process commas :
lines.en=lines.en.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))
lines.de=lines.de.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))

# Getting rid of punctuation
exclude = set(string.punctuation)
lines.en=lines.en.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.de=lines.de.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Getting rid of digits
remove_digits = str.maketrans('', '', digits)
lines.en=lines.en.apply(lambda x: x.translate(remove_digits))
lines.de=lines.de.apply(lambda x: x.translate(remove_digits))

# shit2

In [4]:
# Appending SOS andEOS to target data : 
lines.de = lines.de.apply(lambda x : 'SOS_ '+ x + ' _EOS')

# Create word dictionaries :
en_words=set()
for line in lines.en:
    for word in line.split():
        if word not in en_words:
            en_words.add(word)
    
de_words=set()
for line in lines.de:
    for word in line.split():
        if word not in de_words:
            de_words.add(word)
            
# get lengths and sizes :
num_en_words = len(en_words)
num_de_words = len(de_words)

max_en_words_per_sample = max([len(sample.split()) for sample in lines.en])+5
max_de_words_per_sample = max([len(sample.split()) for sample in lines.de])+5

num_en_samples = len(lines.en)
num_de_samples = len(lines.de)

# Get lists of words :
input_words = sorted(list(en_words))
target_words = sorted(list(de_words))

en_token_to_int = dict()
en_int_to_token = dict()

de_token_to_int = dict()
de_int_to_token = dict()

#Tokenizing the words ( Convert them to numbers ) :
for i,token in enumerate(input_words):
    en_token_to_int[token] = i
    en_int_to_token[i]     = token

for i,token in enumerate(target_words):
    de_token_to_int[token] = i
    de_int_to_token[i]     = token

# initiate numpy arrays to hold the data that our seq2seq model will use:
encoder_input_data = np.zeros(
    (num_en_samples, max_en_words_per_sample),
    dtype='float32')
decoder_input_data = np.zeros(
    (num_de_samples, max_de_words_per_sample),
    dtype='float32')
decoder_target_data = np.zeros(
    (num_de_samples, max_de_words_per_sample, num_de_words),
    dtype='float32')

# Process samples, to get input, output, target data:
for i, (input_text, target_text) in enumerate(zip(lines.en, lines.de)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = en_token_to_int[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = de_token_to_int[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, de_token_to_int[word]] = 1.


# shit 3

In [5]:
#Embedding_layer = Embedding(input_dim = num_words, output_dim = vec_len)

# Building the encoder 

In [6]:
# Defining some constants: 
vec_len       = 300   # Length of the vector that we willl get from the embedding layer
latent_dim    = 1024  # Hidden layers dimension 
dropout_rate  = 0.2   # Rate of the dropout layers
batch_size    = 64    # Batch size
epochs        = 30    # Number of epochs

# Define an input sequence and process it.
# Input layer of the encoder :
encoder_input = Input(shape=(None,))

# Hidden layers of the encoder :
encoder_embedding = Embedding(input_dim = num_en_words, output_dim = vec_len)(encoder_input)
encoder_dropout   = (TimeDistributed(Dropout(rate = dropout_rate)))(encoder_embedding)
encoder_LSTM      = LSTM(latent_dim, return_sequences=True)(encoder_dropout)

# Output layer of the encoder :
encoder_LSTM2_layer = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_LSTM2_layer(encoder_LSTM)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Building the decoder

In [7]:
# Set up the decoder, using `encoder_states` as initial state.
# Input layer of the decoder :
decoder_input = Input(shape=(None,))

# Hidden layers of the decoder :
decoder_embedding_layer = Embedding(input_dim = num_de_words, output_dim = vec_len)
decoder_embedding = decoder_embedding_layer(decoder_input)

decoder_dropout_layer = (TimeDistributed(Dropout(rate = dropout_rate)))
decoder_dropout = decoder_dropout_layer(decoder_embedding)

decoder_LSTM_layer = LSTM(latent_dim, return_sequences=True)
decoder_LSTM = decoder_LSTM_layer(decoder_dropout, initial_state = encoder_states)

decoder_LSTM_2_layer = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_LSTM_2,_,_ = decoder_LSTM_2_layer(decoder_LSTM)

# Output layer of the decoder :
decoder_dense = Dense(num_de_words, activation='softmax')
decoder_outputs = decoder_dense(decoder_LSTM_2)

# Bringing the encoder and decoder together into one model

In [8]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_input, decoder_input], decoder_outputs)

model.summary()

# Define a checkpoint callback :
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    4270200     input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, None, 300)    0           embedding_1[0][0]                
__________________________________________________________________________________________________
embedding_

# Training the Model

In [ ]:
num_train_samples = 9000
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data[:num_train_samples,:],
               decoder_input_data[:num_train_samples,:]],
               decoder_target_data[:num_train_samples,:,:],
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.08,
          callbacks = callbacks_list)

Train on 8280 samples, validate on 720 samples
Epoch 1/30
8280/8280 [==============================] - 2199s 266ms/step - loss: 0.3833 - val_loss: 0.3891

Epoch 00001: val_loss improved from inf to 0.38908, saving model to Weights-001--0.38908.hdf5


/usr/local/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_3 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_2/while/Exit_2:0' shape=(?, 1024) dtype=float32>, <tf.Tensor 'lstm_2/while/Exit_3:0' shape=(?, 1024) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Epoch 2/30
5760/8280 [===================>..........] - ETA: 11:28 - loss: 0.3219